# monolog/KoELECTRA-base

In [21]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12916159188911773792
xla_global_id: -1
]


In [22]:
# # 파이썬 환경 구축 후 라이브러리 버젼 맞춰 설치
# !python -m pip install --upgrade pip
# !pip install transformers==4.34.1 torch==2.1.0 soynlp==0.0.493 requests==2.31.0 tensorflow==2.10.0 accelerate==0.23.0 PyKoSpacing==0.5 kss==4.5.4 matplotlib==3.7.3 wordcloud==1.9.2

# # 파이썬 라이브러리 버젼 전체 확인
!pip freeze

# # 파이썬 라이브러리 버젼 하나씩 확인
# !pip show transformers      # 4.34.1
# !pip show torch             # 2.1.0
# !pip show soynlp            # 0.0.493
# !pip show requests          # 2.31.0                
# !pip show tensorflow        # 2.9.3
# !pip show accelerate        # 0.23.0
# !pip show PyKoSpacing       # 0.5
# !pip show kss               # 4.5.4
# !pip show matplotlib        # 3.7.3
# !pip show wordcloud         # 1.9.2

# git에서 복제 라이브러리
# !git clone https://github.com/ZIZUN/korean-malicious-comments-dataset.git
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

absl-py==2.0.0
asttokens==2.4.1
astunparse==1.6.3
backcall==0.2.0
cachetools==5.3.2
certifi==2023.7.22
charset-normalizer==3.3.1
click==8.1.7
colorama==0.4.6
comm==0.1.4
contourpy==1.1.1
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
emoji==0.6.0
et-xmlfile==1.1.0
executing==2.0.1
filelock==3.13.0
flatbuffers==23.5.26
fonttools==4.43.1
fsspec==2023.10.0
gast==0.4.0
google-auth==2.23.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.59.0
h5py==3.1.0
huggingface-hub==0.18.0
idna==3.4
importlib-metadata==6.8.0
importlib-resources==6.1.0
ipykernel==6.26.0
ipython==8.12.3
jedi==0.19.1
joblib==1.3.2
JPype1==1.4.1
jupyter_client==8.5.0
jupyter_core==5.4.0
keras==2.11.0
Keras-Preprocessing==1.1.2
kiwisolver==1.4.5
libclang==16.0.6
Markdown==3.5
MarkupSafe==2.1.3
matplotlib==3.7.3
matplotlib-inline==0.1.6
nest-asyncio==1.5.8
numpy==1.24.4
oauthlib==3.2.2
openpyxl==3.1.2
opt-einsum==3.3.0
packaging==23.2
pandas==2.0.3
parso==0.8.3
pickleshare==0.7.5
Pillow==10.1.0
platformdirs==3.11.0


In [23]:
!pip install wordcloud==1.9.2

In [24]:
import re
import torch
import pandas as pd

# deivce 선택
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("devcie:", device)

devcie: cpu


In [25]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\gjaischool1\Desktop\프로젝트 코드\Sentence data_pra.xlsx")

# 'Sentence' 칼럼의 값을 문자열로 변환하여 리스트로 저장
str_data = df['Sentence'].astype(str).tolist()

# 'Sentence' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(str_data)

print(df.shape)
df.head()

(1830, 2)


,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [27]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [28]:
train_data = df.sample(frac=0.8, random_state=42)                 # train(80%), test(20%) 셋 구분 
test_data = df.drop(train_data.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["Sentence"], inplace=True)
test_data.drop_duplicates(subset=["Sentence"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋 : 1464
중복 제거 전 테스트 데이터셋 : 366
중복 제거 후 학습 데이터셋 : 1464
중복 제거 후 테스트 데이터셋 : 366


In [29]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import ElectraModel
import torch

# KoELECTRA 모델 및 토크나이저 로드
model_name = "monologg/koelectra-base-discriminator"
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

loading file https://huggingface.co/monologg/koelectra-base-discriminator/resolve/main/vocab.txt from cache at C:\Users\gjaischool1/.cache\huggingface\transformers\7e08529d58c21011d83413d150919bd61c712ecd0ceb2619058f2e0a99eb98bf.42b747a439e42193b1b54614a2e0c4608d8ec136b7405c799d29f667ccdbb27d
loading file https://huggingface.co/monologg/koelectra-base-discriminator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/monologg/koelectra-base-discriminator/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/monologg/koelectra-base-discriminator/resolve/main/tokenizer_config.json from cache at C:\Users\gjaischool1/.cache\huggingface\transformers\d64856f5e184ded19a7270a244bb101fbdb72b0ea6f87249c37d9e5804bbb3ce.767d1b760a83978bae6c324157fad57ee513af333a7cea6986e852579f6f0dd1
loading file https://huggingface.co/monologg/koelectra-base-discriminator/resolve/main/tokenizer.json from cache at None
loading configuration fil

In [30]:
tokenized_train_sentences = tokenizer(
    list(train_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [31]:
tokenized_test_sentences = tokenizer(
    list(test_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [32]:
# print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체 
# print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
# print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

In [33]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, labels):
        self.encodings = encoding
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [34]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)   # 사전 학습된 모델 찾아오기
model.to(device)

loading configuration file https://huggingface.co/monologg/koelectra-base-discriminator/resolve/main/config.json from cache at C:\Users\gjaischool1/.cache\huggingface\transformers\08392979cd0a255c73391b0f98f2a7542f25962bc67e3025f7a13b37faf6ab8c.2939fcc03237a86d77e94b618ccc8dd10735c12dc5287ff966be6040b83bf322
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.11.3",
  "type_vocab_size"

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32200, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [36]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=500,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [37]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [38]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [39]:
trainer.train()

***** Running training *****
  Num examples = 1464
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1830


C:\Users\gjaischool1\AppData\Local\Temp\ipykernel_16140\1167134061.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여


KeyboardInterrupt: 

In [ ]:
# 평가를 수행하고 custom_ 접두사가 붙은 평가 지표를 출력
results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='custom_')

print(results)

C:\Users\user\AppData\Local\Temp\ipykernel_3880\1167134061.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여


  0%|          | 0/1 [00:00<?, ?it/s]

{'custom__loss': 0.9143447279930115, 'custom__accuracy': 0.782608695652174, 'custom__f1': 0.7058823529411764, 'custom__precision': 1.0, 'custom__recall': 0.5454545454545454, 'custom__runtime': 2.9918, 'custom__samples_per_second': 7.688, 'custom__steps_per_second': 0.334, 'epoch': 10.0}


In [ ]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    # tokenized_sent.to(deivce)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    sentence = True
    input_sentence = tokenized_sent["input_ids"]
    if result == 0:
        sentence = True

    elif result == 1:
        sentence = False
    return sentence, input_sentence


sentence_predict("씨발이게 맞아?")

(True, tensor([[2295,  640, 3898, 5170,   33]]))

In [ ]:
def found_word(input_sentence, found_bad_word):
    result = input_sentence
    if found_bad_word:
        result = badword_find(result)
    return result

def badword_find(input_sentence):
    result = input_sentence
    badword_df = pd.read_excel(r'C:\Users\user\Desktop\X-filter\word_list.xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in input_sentence:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
            else:
                result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

# 테스트
input_sentence = "엄마가 죽었으면 좋겠어"
speak_result = badword_find(input_sentence)
print(speak_result)

혐오 표현입니다.


In [ ]:
badword_find(input_sentence)

In [ ]:
def speak(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == False:
        return sent
    elif sentence[0] == True:
        return badword_find(sent)

In [ ]:
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [ ]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [ ]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           #  제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("아버지가방에들어가신다")

'아버지가 방에 들어가신다'

In [ ]:
def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        text_output = speak(sentence)
        sentences_list.append(text_output)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [ ]:
final_output()